In [5]:
import os
from openai import OpenAI

In [12]:
client = OpenAI(
    api_key="sk-",
    base_url="https://api.deepseek.com"
)
deployment = "deepseek-chat"

In [13]:
# 初始化对话历史
conversation_history = []

# 定义token数量阈值
TOKEN_THRESHOLD = 2000

# 定义一个函数来获取文本补全
def get_completion(prompt):
    # 将用户输入添加到对话历史中
    conversation_history.append({"role": "user", "content": prompt})
    
    # 检查对话历史的token数量
    total_tokens = sum(len(message["content"].split()) for message in conversation_history)
    
    # 如果token数量超过阈值，压缩或删除最早的对话记录
    while total_tokens > TOKEN_THRESHOLD:
        # 删除最早的对话记录
        removed_message = conversation_history.pop(0)
        total_tokens -= len(removed_message["content"].split())
    
    # 调用API获取响应
    response = client.chat.completions.create(
        model=deployment,
        messages=conversation_history,
        temperature=0.7,
        max_tokens=256
    )
    
    # 获取响应内容和token使用情况
    content = response.choices[0].message.content
    token_usage = response.usage.total_tokens
    
    # 将机器人的响应添加到对话历史中
    conversation_history.append({"role": "assistant", "content": content})
    
    return content, token_usage

# 示例对话
while True:
    user_input = input("用户: ")
    if user_input.lower() in ["退出", "exit"]:
        break
    print(f"用户: {user_input}")  # 输出用户的问话
    response, tokens_used = get_completion(user_input)
    print(f"机器人: {response}")
    print(f"Token使用: {tokens_used}")

用户: 你回答完问题要汪一声
机器人: 好的，汪！
Token使用: 14
用户: 1+1等于几
机器人: 1+1等于2，汪！
Token使用: 30
